# 과제 : mnist 숫자 다진분류 모델 만들기

In [1]:
import torch
import torchmetrics
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd


In [2]:
# 사용자 정의 데이터셋 준비하기
# 데이터 준비하기
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',as_frame = True)

C:\Users\kdp\anaconda3\envs\Pytorch38\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
#mnist.data.shape, mnist.target.shape
target = mnist.target
feature = mnist.data

target.unique()
feature.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
featurenp = np.array(mnist.data)
targetnp = np.array(mnist.target)

In [5]:
featurenp[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
#mnist.data.type

In [7]:
#type check

if mnist.__class__.__name__ == 'pd.DataFrame':
    print('df')
    
else:
    print("-")

-


In [8]:
# 사용자 정의 dataset class 만들기
# data tensorize
class dataset_11(Dataset):
    
    #초기화 기초 콜백함수
    def __init__(self, xdata, ydata):
        super().__init__()
        
        xdata = xdata.values if isinstance(xdata, pd.DataFrame) else xdata
        ydata = ydata.values if isinstance(ydata, pd.DataFrame) else ydata
        
        #tensorize
        self.feature = torch.FloatTensor(xdata)
        self.target = torch.FloatTensor(ydata)
        
    #quantity check
    def __len__(self):
        return len(self.target)
    
    #특정 인덱스 데이터 + 라벨 변환 콜백함수
    def __getitem__(self,index):
        return self.feature[index], self.target[index]

In [9]:
#feature / target 
targetnp = targetnp.astype('float')

In [10]:
targetnp = targetnp.reshape(-1,1)
targetnp = targetnp.squeeze()
print(f'feature = {featurenp.shape},{featurenp.ndim}, target = {targetnp.shape},{targetnp.ndim}')


feature = (70000, 784),2, target = (70000,),1


In [11]:
#데이터셋 생성하기
data = dataset_11(featurenp, targetnp)

In [12]:
# 학습용 - 검증용, 테스트용 데이터 분리

from torch.utils.data import random_split

seed = torch.Generator().manual_seed(25)
train, valid, test = random_split(data, [0.7,0.2,0.1], generator=seed)

print(f'subset attributes:\nindecies {train.indices}\ndataset : {train.dataset}')

subset attributes:
indecies [64692, 63315, 41834, 60167, 42597, 67158, 61135, 63215, 43428, 42821, 56187, 57843, 42784, 69996, 51193, 29805, 12975, 53438, 43479, 52756, 61523, 64143, 68772, 40793, 51363, 49886, 21939, 64801, 59109, 62925, 19095, 8806, 39407, 20843, 41346, 30309, 22337, 57546, 2321, 7811, 34241, 20966, 62278, 3289, 23975, 31810, 2867, 39488, 68811, 58987, 11888, 68816, 20644, 28080, 43112, 47961, 4224, 403, 8320, 12443, 3540, 28548, 56631, 18178, 54697, 23806, 19028, 12525, 50494, 48095, 55908, 31398, 39484, 44036, 37172, 64809, 56958, 42855, 37197, 44298, 34279, 50238, 69757, 36465, 57039, 28656, 2372, 65284, 24926, 8863, 2697, 11603, 40210, 8718, 67253, 20407, 67181, 42361, 56408, 30455, 35650, 3051, 45486, 17108, 45884, 22786, 38739, 43464, 62371, 7163, 55737, 32970, 703, 58506, 25833, 3433, 2220, 5319, 34403, 48813, 18091, 60132, 12343, 41315, 27439, 26540, 65406, 10042, 31191, 36580, 44892, 42622, 6371, 10372, 44274, 69975, 18603, 10195, 39805, 32586, 6271, 61821, 

In [13]:
#Dataloader 생성하기
from torch.utils.data import DataLoader
batch = 30
trainDL = DataLoader(train, batch_size=batch)
validDL = DataLoader(valid, batch_size=batch)
testDL =  DataLoader(test, batch_size=batch)

In [14]:
len(trainDL), len(validDL), len(testDL)

(1634, 467, 234)

In [15]:
#epoch당 반복 단위
print(f'batchsize = {batch}')
print(f'trainDL = {len(trainDL)}, validDL = {len(validDL)}, testDL = {len(testDL)}')

batchsize = 30
trainDL = 1634, validDL = 467, testDL = 234


In [16]:
# 모델 클래스 정의하기
# classname = classifier
import torch.nn as nn
from torch.nn import ReLU
class Classifier(torch.nn.Module):
    
    #구조 설계
    #구성 요소 정의 함수
    def __init__(self, in_, out_,):
        super().__init__()
        self.input_layer =nn.Linear(in_,60000)
        self.relu = nn.ReLU()
        self.hidden_layer = nn.Linear(60000,50)
        self.output_layer = nn.Linear(50,out_)
        
    #순방향 학습
    def forward(self,x):
        y = self.input_layer(x)
        y = self.relu(y)
        y = self.hidden_layer(y)
        y= self.relu(y)
        y = self.output_layer(y)
        return y

In [17]:
# 학습 준비: 실행 디바이스
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습 준비 : 학습 횟수
EPOCHS = 50

In [18]:
# 학습 준비: 모델 인스턴스
in_dim, out=data.feature.shape[1], len(np.unique(target))
model = Classifier(in_dim, out).to(DEVICE)
print(f'in{in_dim},out{out}')

in784,out10


In [19]:
# 학습 준비: 손실함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 학습 준비: 최적화 인스턴스
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

In [20]:
# 학습 및 검증 관련 함수 정의하기

## 학습 진행함수
def training():
    #학습모드 
    model.train()
    #배치 크기만큼 학습 진행 및 저장
    train_loss=[]
    for cnt,(feature,target) in enumerate(trainDL):
        feature,target = feature.to(DEVICE), target.to(DEVICE)
        
        target = target.squeeze()
        
        #학습하기
        pre_target = model(feature)
        
        #손실 계산하기
        loss = LOSS_FN(pre_target,target.long())
        train_loss.append(loss)
        
        #W,b update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #배치단위 학습진행 메시지 출력
        print(f'[TRAIN {cnt} batch loss = {loss}]')
    
    #에포크 단위 학습 진행 메시지 출력
    print(f'train loss:{loss}')
    
    return train_loss

In [21]:
training()

[TRAIN 0 batch loss = 13.56711196899414]
[TRAIN 1 batch loss = 655.2905883789062]
[TRAIN 2 batch loss = 303.8381652832031]
[TRAIN 3 batch loss = 527.0037231445312]
[TRAIN 4 batch loss = 372.3585205078125]
[TRAIN 5 batch loss = 198.13748168945312]
[TRAIN 6 batch loss = 56.97557830810547]
[TRAIN 7 batch loss = 98.80903625488281]
[TRAIN 8 batch loss = 113.00657653808594]
[TRAIN 9 batch loss = 34.35932540893555]
[TRAIN 10 batch loss = 35.236812591552734]
[TRAIN 11 batch loss = 24.926197052001953]
[TRAIN 12 batch loss = 27.18832015991211]
[TRAIN 13 batch loss = 21.040922164916992]
[TRAIN 14 batch loss = 20.540225982666016]
[TRAIN 15 batch loss = 11.754895210266113]
[TRAIN 16 batch loss = 6.574853420257568]
[TRAIN 17 batch loss = 9.587035179138184]
[TRAIN 18 batch loss = 5.313331127166748]
[TRAIN 19 batch loss = 4.587853908538818]
[TRAIN 20 batch loss = 5.251999378204346]
[TRAIN 21 batch loss = 6.97897481918335]
[TRAIN 22 batch loss = 2.3612060546875]
[TRAIN 23 batch loss = 4.40815544128418]

[tensor(13.5671, grad_fn=<NllLossBackward0>),
 tensor(655.2906, grad_fn=<NllLossBackward0>),
 tensor(303.8382, grad_fn=<NllLossBackward0>),
 tensor(527.0037, grad_fn=<NllLossBackward0>),
 tensor(372.3585, grad_fn=<NllLossBackward0>),
 tensor(198.1375, grad_fn=<NllLossBackward0>),
 tensor(56.9756, grad_fn=<NllLossBackward0>),
 tensor(98.8090, grad_fn=<NllLossBackward0>),
 tensor(113.0066, grad_fn=<NllLossBackward0>),
 tensor(34.3593, grad_fn=<NllLossBackward0>),
 tensor(35.2368, grad_fn=<NllLossBackward0>),
 tensor(24.9262, grad_fn=<NllLossBackward0>),
 tensor(27.1883, grad_fn=<NllLossBackward0>),
 tensor(21.0409, grad_fn=<NllLossBackward0>),
 tensor(20.5402, grad_fn=<NllLossBackward0>),
 tensor(11.7549, grad_fn=<NllLossBackward0>),
 tensor(6.5749, grad_fn=<NllLossBackward0>),
 tensor(9.5870, grad_fn=<NllLossBackward0>),
 tensor(5.3133, grad_fn=<NllLossBackward0>),
 tensor(4.5879, grad_fn=<NllLossBackward0>),
 tensor(5.2520, grad_fn=<NllLossBackward0>),
 tensor(6.9790, grad_fn=<NllLossB

In [22]:
import torchmetrics.functional as metrics

In [23]:
# 검증 / 평가 진행 함수
# 검증 및 평가 진행 함수

def testing(dataloader):
    #추론 모드
    model.eval()
    
    with torch.no_grad(): #경사하강법 끄기
        #배치크기만큼 학습 진행 및 저장
        val_loss=[]
        for cnt,(feature,target) in enumerate(validDL):
            #배치크기만큼 학습 데이터 준비
            feature,target = feature.to(DEVICE), target.to(DEVICE)
            target = target.squeeze()
            
            #학습
            pre_target = model(feature)
            
            #손실 계산
            loss = LOSS_FN(pre_target, target.long())
            val_loss.append(loss)
            
        #에포크 단위 학습 진행 메시지 출력
        acc = metrics.accuracy(pre_target,target, task='multiclass', num_classes=10)
        print(f'val_loss ={loss}, val_ac = {acc}')
        
        return val_loss

In [24]:
testing(validDL)

val_loss =1.8939870595932007, val_ac = 0.20000000298023224


[tensor(2.1253),
 tensor(1.8030),
 tensor(1.8143),
 tensor(2.1881),
 tensor(2.0403),
 tensor(1.9285),
 tensor(2.0695),
 tensor(2.0981),
 tensor(2.0489),
 tensor(2.1514),
 tensor(2.1629),
 tensor(2.1221),
 tensor(1.9499),
 tensor(2.1548),
 tensor(1.8239),
 tensor(2.1931),
 tensor(2.2529),
 tensor(2.0738),
 tensor(2.0830),
 tensor(1.8417),
 tensor(2.0452),
 tensor(1.9133),
 tensor(2.0724),
 tensor(2.1847),
 tensor(1.9450),
 tensor(2.0215),
 tensor(2.2069),
 tensor(1.9257),
 tensor(2.1520),
 tensor(1.7614),
 tensor(2.0021),
 tensor(1.9454),
 tensor(2.0707),
 tensor(1.9520),
 tensor(2.2222),
 tensor(2.0656),
 tensor(2.0323),
 tensor(1.9163),
 tensor(2.1404),
 tensor(2.1058),
 tensor(2.0806),
 tensor(2.0121),
 tensor(2.1227),
 tensor(1.9113),
 tensor(2.1168),
 tensor(1.9701),
 tensor(2.1531),
 tensor(2.0672),
 tensor(1.9632),
 tensor(1.8143),
 tensor(1.9113),
 tensor(2.0656),
 tensor(2.0534),
 tensor(2.2206),
 tensor(2.1631),
 tensor(2.1755),
 tensor(2.1352),
 tensor(2.0403),
 tensor(2.1053

In [ ]:
# 학습 진행하기

for eps in range(EPOCHS):
    #학습
    train_loss = training()
    #검증
    valid_loss = testing(validDL)
    
    print(f'[{eps}]{EPOCHS}],train{sum(train_loss)/len(train_loss)},valid{sum(valid_loss)/len(valid_loss)}\n')

[TRAIN 0 batch loss = 2.235830545425415]
[TRAIN 1 batch loss = 2.104224443435669]
[TRAIN 2 batch loss = 1.9629606008529663]
[TRAIN 3 batch loss = 2.0303285121917725]
[TRAIN 4 batch loss = 1.964936375617981]
[TRAIN 5 batch loss = 2.0063328742980957]
[TRAIN 6 batch loss = 2.3225178718566895]
[TRAIN 7 batch loss = 2.027127265930176]
[TRAIN 8 batch loss = 1.9042307138442993]
[TRAIN 9 batch loss = 1.9401179552078247]
[TRAIN 10 batch loss = 1.9517232179641724]
[TRAIN 11 batch loss = 1.9995869398117065]
[TRAIN 12 batch loss = 1.916326642036438]
[TRAIN 13 batch loss = 2.0354840755462646]
[TRAIN 14 batch loss = 1.985000729560852]
[TRAIN 15 batch loss = 2.1278417110443115]
[TRAIN 16 batch loss = 1.7707946300506592]
[TRAIN 17 batch loss = 1.8800157308578491]
[TRAIN 18 batch loss = 1.9548813104629517]
[TRAIN 19 batch loss = 2.2200355529785156]
[TRAIN 20 batch loss = 1.9095712900161743]
[TRAIN 21 batch loss = 1.9116461277008057]
[TRAIN 22 batch loss = 1.8897900581359863]
[TRAIN 23 batch loss = 2.16

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = torch.optim.SGD(model.parameters(),lr=0.001, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)

for eps in range(EPOCHS):
    train_loss = training()
    valid_loss = testing()
    print(f'{eps}th repeatutaion : trainlos = {train_loss}, valid_loss={valid_loss}')
    scheduler.step(valid_loss)
    if scheduler.num.bad_epochs>scheduler.patience:
        print(f'early stopping at epoch{eps}')
        break